# かがわAIゼミナール　2020
## 課題
 データセット：「Kickstarter Projects」  
 問題設定：クラウドファンデイングが成功するか(state)を予測する。

 https://www.kaggle.com/kemical/kickstarter-projects  
 データ:ks-projects-201801.csv

In [1]:
%matplotlib inline
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix
from sklearn.metrics import mean_squared_error, mean_absolute_error # 回帰問題における性能評価に関する関数
from  sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix # 回帰問題における性能評価に関する関数

### CSVファイルの中身を確認する｡  

|項目|説明|目的変数|説明変数候補|
|-|-|-|-|
|ID|-|-|-|
|name|-|-|-|
|category|-|-|〇|
|main_category|category of campaign|-|〇|
|currency|currency used to support:通貨|-|〇|
|deadline|deadline for crowdfunding:終了|-|〇|
|goal|目標金額|-|〇|
|launched|date launched:開始|-|〇|
|pledged|amount pledged by "crowd"	集まったお金|-|-|		
|state|結果|〇|-|
|backers|後援者,パトロン|-|-|		
|country|-|-|〇|
|usd pledged|-|-|-|				
|usd_pledged_real|-|-|-|
|usd_goal_real|-|-|-|


In [2]:
#CSVを読み込む
ks_proj = pd.read_csv("../kaggle/ks-projects-201801_csv/ks-projects-201801.csv")[['state','category','main_category','currency','goal','launched','deadline','country']]

In [3]:
#内容確認
ks_proj.info()
display(ks_proj.head())
ks_proj.describe(include='O')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378661 entries, 0 to 378660
Data columns (total 8 columns):
state            378661 non-null object
category         378661 non-null object
main_category    378661 non-null object
currency         378661 non-null object
goal             378661 non-null float64
launched         378661 non-null object
deadline         378661 non-null object
country          378661 non-null object
dtypes: float64(1), object(7)
memory usage: 23.1+ MB


,state,category,main_category,currency,goal,launched,deadline,country
0,failed,Poetry,Publishing,GBP,1000.0,2015-08-11 12:12:28,2015-10-09,GB
1,failed,Narrative Film,Film & Video,USD,30000.0,2017-09-02 04:43:57,2017-11-01,US
2,failed,Narrative Film,Film & Video,USD,45000.0,2013-01-12 00:20:50,2013-02-26,US
3,failed,Music,Music,USD,5000.0,2012-03-17 03:24:11,2012-04-16,US
4,canceled,Film & Video,Film & Video,USD,19500.0,2015-07-04 08:35:03,2015-08-29,US


,state,category,main_category,currency,launched,deadline,country
count,378661,378661,378661,378661,378661,378661,378661
unique,6,159,15,14,378089,3164,23
top,failed,Product Design,Film & Video,USD,1970-01-01 01:00:00,2014-08-08,US
freq,197719,22314,63585,295365,7,705,292627


In [4]:
#stateのunipueが6あるため内容を確認
ks_proj['state'].unique()

array(['failed', 'canceled', 'successful', 'live', 'undefined',
       'suspended'], dtype=object)

今回は'failed', 'successful'のロジスティクス回帰とし、それ以外は削除する。

In [5]:
#ks2を作成
ks2= ks_proj.copy()
#state列が'failed','successful'の行を抽出する。
ks2 = ks2[ks2['state'].isin(['failed','successful'])]
display(ks2.head())
ks2.describe(include='O')

,state,category,main_category,currency,goal,launched,deadline,country
0,failed,Poetry,Publishing,GBP,1000.0,2015-08-11 12:12:28,2015-10-09,GB
1,failed,Narrative Film,Film & Video,USD,30000.0,2017-09-02 04:43:57,2017-11-01,US
2,failed,Narrative Film,Film & Video,USD,45000.0,2013-01-12 00:20:50,2013-02-26,US
3,failed,Music,Music,USD,5000.0,2012-03-17 03:24:11,2012-04-16,US
5,successful,Restaurants,Food,USD,50000.0,2016-02-26 13:38:27,2016-04-01,US


,state,category,main_category,currency,launched,deadline,country
count,331675,331675,331675,331675,331675,331675,331675
unique,2,159,15,14,331255,3102,23
top,failed,Product Design,Film & Video,USD,2016-03-01 02:04:22,2014-08-08,US
freq,197719,18680,56527,261511,2,612,261360


## 説明変数の整理

#### launched,deadlineについて  
launched(開始）とdeadline(終了）の差分をとって期間を抽出する。  
objectからdatetimeに変換する。

In [6]:
##時分秒を削除する
ks2['launched'] =ks2['launched'].str[:11]

##object→datetimeに変更（処理に時間がかかる）
ks2['launched'] = pd.to_datetime(ks2['launched'])
ks2['deadline'] = pd.to_datetime(ks2['deadline'])
ks2['days'] = (ks2['deadline'] - ks2['launched']).dt.days
ks2['days'].head()

0    59
1    60
2    45
3    30
5    35
Name: days, dtype: int64

#### currencyについて

In [7]:
#currencyの内訳を確認する。
ks2['currency'].value_counts()

USD    261511
GBP     29476
EUR     14378
CAD     12375
AUD      6621
SEK      1510
MXN      1411
NZD      1274
DKK       929
CHF       652
NOK       584
HKD       477
SGD       454
JPY        23
Name: currency, dtype: int64

In [8]:
(ks2['currency'].value_counts()/ks2['currency'].count())*100

USD    78.845557
GBP     8.887013
EUR     4.334966
CAD     3.731062
AUD     1.996231
SEK     0.455265
MXN     0.425416
NZD     0.384111
DKK     0.280093
CHF     0.196578
NOK     0.176076
HKD     0.143815
SGD     0.136881
JPY     0.006934
Name: currency, dtype: float64

USD,GBP,EUR,CADで95%を占めているため他は無視する。

'goal'の値をそろえるためUSDに統一する。  
為替レート2018年平均  
EUR→USD　1.18  
GBP→USD　1.33  
CAD→USD　0.77  
https://fxtop.com/jp/cnv.php

In [9]:
#currency列が'USD','GBP','EUR'の行を抽出する。
ks2 = ks2[ks2['currency'].isin(['USD','GBP','EUR','CAD'])]

In [10]:
ks2.loc[ks2['currency'] == 'USD', 'goal'] = ks2['goal']
ks2.loc[ks2['currency'] == 'GBP', 'goal'] = ks2['goal']*1.33
ks2.loc[ks2['currency'] == 'EUR', 'goal'] = ks2['goal']*1.18
ks2.loc[ks2['currency'] == 'CAD', 'goal'] = ks2['goal']*0.77
display(ks2.head())

,state,category,main_category,currency,goal,launched,deadline,country,days
0,failed,Poetry,Publishing,GBP,1330.0,2015-08-11,2015-10-09,GB,59
1,failed,Narrative Film,Film & Video,USD,30000.0,2017-09-02,2017-11-01,US,60
2,failed,Narrative Film,Film & Video,USD,45000.0,2013-01-12,2013-02-26,US,45
3,failed,Music,Music,USD,5000.0,2012-03-17,2012-04-16,US,30
5,successful,Restaurants,Food,USD,50000.0,2016-02-26,2016-04-01,US,35


In [11]:
#currencyをダミー変数に変換
ks3 = ks2.copy()
crcy = pd.get_dummies(ks2['currency'],dtype=int)
ks3 = pd.concat([ks3, crcy], axis=1)
display(ks3.head())

,state,category,main_category,currency,goal,launched,deadline,country,days,CAD,EUR,GBP,USD
0,failed,Poetry,Publishing,GBP,1330.0,2015-08-11,2015-10-09,GB,59,0,0,1,0
1,failed,Narrative Film,Film & Video,USD,30000.0,2017-09-02,2017-11-01,US,60,0,0,0,1
2,failed,Narrative Film,Film & Video,USD,45000.0,2013-01-12,2013-02-26,US,45,0,0,0,1
3,failed,Music,Music,USD,5000.0,2012-03-17,2012-04-16,US,30,0,0,0,1
5,successful,Restaurants,Food,USD,50000.0,2016-02-26,2016-04-01,US,35,0,0,0,1


#### countryについて
currencyとほぼ同じと思われるため説明変数にしない。

In [12]:
ks2['country'].value_counts()

US      261360
GB       29454
CA       12370
DE        3436
FR        2520
NL        2411
IT        2369
ES        1873
IE         683
BE         523
AT         485
N,0"       199
LU          57
Name: country, dtype: int64

In [13]:
ks2['currency'].value_counts()

USD    261511
GBP     29476
EUR     14378
CAD     12375
Name: currency, dtype: int64

#### category,main_categoryについて
categoryは159種、main_categoryは15種類のためmain_categoryを説明変数として採用する。

In [14]:
ks2['main_category'].value_counts()

Film & Video    54858
Music           44892
Publishing      34006
Games           26995
Technology      24958
Art             24736
Design          23651
Food            21222
Fashion         18644
Theater         10044
Comics           9599
Photography      9271
Crafts           7447
Journalism       3920
Dance            3497
Name: main_category, dtype: int64

In [15]:
m_ctg = pd.get_dummies(ks2['main_category'],dtype=int)
ks3 = pd.concat([ks3, m_ctg], axis=1)
display(ks3.head())

,state,category,main_category,currency,goal,launched,deadline,country,days,CAD,...,Fashion,Film & Video,Food,Games,Journalism,Music,Photography,Publishing,Technology,Theater
0,failed,Poetry,Publishing,GBP,1330.0,2015-08-11,2015-10-09,GB,59,0,...,0,0,0,0,0,0,0,1,0,0
1,failed,Narrative Film,Film & Video,USD,30000.0,2017-09-02,2017-11-01,US,60,0,...,0,1,0,0,0,0,0,0,0,0
2,failed,Narrative Film,Film & Video,USD,45000.0,2013-01-12,2013-02-26,US,45,0,...,0,1,0,0,0,0,0,0,0,0
3,failed,Music,Music,USD,5000.0,2012-03-17,2012-04-16,US,30,0,...,0,0,0,0,0,1,0,0,0,0
5,successful,Restaurants,Food,USD,50000.0,2016-02-26,2016-04-01,US,35,0,...,0,0,1,0,0,0,0,0,0,0


In [16]:
#'state'をTrue/Falseに変換する。
ks3['state'] = ks3['state'] == 'successful' 
display(ks3.head())

,state,category,main_category,currency,goal,launched,deadline,country,days,CAD,...,Fashion,Film & Video,Food,Games,Journalism,Music,Photography,Publishing,Technology,Theater
0,False,Poetry,Publishing,GBP,1330.0,2015-08-11,2015-10-09,GB,59,0,...,0,0,0,0,0,0,0,1,0,0
1,False,Narrative Film,Film & Video,USD,30000.0,2017-09-02,2017-11-01,US,60,0,...,0,1,0,0,0,0,0,0,0,0
2,False,Narrative Film,Film & Video,USD,45000.0,2013-01-12,2013-02-26,US,45,0,...,0,1,0,0,0,0,0,0,0,0
3,False,Music,Music,USD,5000.0,2012-03-17,2012-04-16,US,30,0,...,0,0,0,0,0,1,0,0,0,0
5,True,Restaurants,Food,USD,50000.0,2016-02-26,2016-04-01,US,35,0,...,0,0,1,0,0,0,0,0,0,0


In [17]:
ks3['state'].value_counts()

False    188261
True     129479
Name: state, dtype: int64

In [18]:
ks3.columns

Index(['state', 'category', 'main_category', 'currency', 'goal', 'launched',
       'deadline', 'country', 'days', 'CAD', 'EUR', 'GBP', 'USD', 'Art',
       'Comics', 'Crafts', 'Dance', 'Design', 'Fashion', 'Film & Video',
       'Food', 'Games', 'Journalism', 'Music', 'Photography', 'Publishing',
       'Technology', 'Theater'],
      dtype='object')

In [19]:
#不要な目的変数は削除する。
ks3 = ks3.drop([ 'category', 'main_category','currency','launched','deadline', 'country'],axis=1)

In [20]:
ks3.dtypes

state              bool
goal            float64
days              int64
CAD               int32
EUR               int32
GBP               int32
USD               int32
Art               int32
Comics            int32
Crafts            int32
Dance             int32
Design            int32
Fashion           int32
Film & Video      int32
Food              int32
Games             int32
Journalism        int32
Music             int32
Photography       int32
Publishing        int32
Technology        int32
Theater           int32
dtype: object

***
ロジスティック回帰により予測する。

In [21]:
y = ks3['state'].values
X = ks3.drop('state', axis=1).values
clf = SGDClassifier(loss='log', penalty='none', max_iter=10000, fit_intercept=True, random_state=1234, tol=1e-3)
clf.fit(X, y)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=10000,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='none',
       power_t=0.5, random_state=1234, shuffle=True, tol=0.001,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [22]:
# ラベルを予測
y_est = clf.predict(X)

# 対数尤度を表示
print('対数尤度 = {:.3f}'.format(- log_loss(y, y_est)))

# 正答率を表示
print('正答率 = {:.3f}%'.format(100 * accuracy_score(y, y_est)))

対数尤度 = -14.026
正答率 = 59.392%


In [23]:
# 正答率を計算
accuracy =  accuracy_score(y, y_est)
print('正答率（Accuracy） = {:.3f}%'.format(100 * accuracy))

# Precision, Recall, F1-scoreを計算
precision, recall, f1_score, _ = precision_recall_fscore_support(y, y_est)

# カテゴリ「2000万以上」に関するPrecision, Recall, F1-scoreを表示
print('適合率（Precision） = {:.3f}%'.format(100 * precision[1]))
print('再現率（Recall） = {:.3f}%'.format(100 * recall[1]))
print('F1値（F1-score） = {:.3f}%'.format(100 * f1_score[1]))

正答率（Accuracy） = 59.392%
適合率（Precision） = 66.766%
再現率（Recall） = 0.694%
F1値（F1-score） = 1.373%


In [24]:
# 予測値と正解のクロス集計（混同行列）
conf_mat = confusion_matrix(y, y_est)
conf_mat = pd.DataFrame(conf_mat, 
                        index=['正解 = failed', '正解 = successful'], 
                        columns=['予測 = failed', '予測 = successful'])
conf_mat

,予測 = failed,予測 = successful
正解 = failed,187814,447
正解 = successful,128581,898
